## 1. Import and Install Dependencies

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import time
import os
import mediapipe as mp

## 2.KeyPoints Using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic      # Holistic Model
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities
mp_drawing_styles = mp.solutions.drawing_styles 
mp_face_mesh = mp.solutions.face_mesh

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION, landmark_drawing_spec=None,
                                  connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()) # Draw face connections
        
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                  landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [6]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()   # read feed

        # Make prediction
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks
        draw_landmarks(image, results)
        
        cv2.imshow('Video to Detect', image)     # show to screen
        if cv2.waitKey(10) & 0xFF==ord('q'):     # break gracefully
            break
            
    cap.release()
    cv2.destroyAllWindows()

## 3. Extract Keypoint Values

In [7]:
print(results.face_landmarks)

landmark {
  x: 0.633335114
  y: 0.688372552
  z: -0.027238423
}
landmark {
  x: 0.647171617
  y: 0.629310489
  z: -0.0573253296
}
landmark {
  x: 0.636185229
  y: 0.644835949
  z: -0.0287105683
}
landmark {
  x: 0.633713722
  y: 0.569399893
  z: -0.0491563082
}
landmark {
  x: 0.64945
  y: 0.612240672
  z: -0.0618983768
}
landmark {
  x: 0.64825213
  y: 0.588382483
  z: -0.0587382726
}
landmark {
  x: 0.641067445
  y: 0.52882117
  z: -0.0321681164
}
landmark {
  x: 0.53763473
  y: 0.511381388
  z: -0.015774
}
landmark {
  x: 0.640579939
  y: 0.484116048
  z: -0.0264907107
}
landmark {
  x: 0.642565966
  y: 0.462041289
  z: -0.0294233989
}
landmark {
  x: 0.642985165
  y: 0.392474651
  z: -0.0225978661
}
landmark {
  x: 0.632153273
  y: 0.696706474
  z: -0.0248737838
}
landmark {
  x: 0.630305588
  y: 0.703112841
  z: -0.0204717014
}
landmark {
  x: 0.628012121
  y: 0.705725968
  z: -0.0154070044
}
landmark {
  x: 0.628369451
  y: 0.706546426
  z: -0.0149667338
}
landmark {
  x: 0.6286

In [8]:
pose = []

for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

print(len(pose))
print(np.ndim(pose))
print(np.shape(pose))

print(pose)

33
2
(33, 4)
[array([ 0.62523401,  0.63355798, -1.69977474,  0.99931407]), array([ 0.6611461 ,  0.53973854, -1.62931979,  0.99854416]), array([ 0.68557674,  0.53780961, -1.62955594,  0.99873167]), array([ 0.70567667,  0.53653812, -1.62988889,  0.99789143]), array([ 0.58432096,  0.54072356, -1.64942765,  0.99868095]), array([ 0.55775529,  0.54101568, -1.64927161,  0.99899012]), array([ 0.53228772,  0.54370511, -1.64963126,  0.9986971 ]), array([ 0.72557038,  0.55531919, -1.16648042,  0.99819607]), array([ 0.47277302,  0.57346892, -1.23379028,  0.99926233]), array([ 0.66057599,  0.7231496 , -1.51708651,  0.99901843]), array([ 0.5767374 ,  0.72607887, -1.53939688,  0.99938309]), array([ 0.86371136,  0.97116429, -0.87182009,  0.97476733]), array([ 0.32078588,  0.99813187, -0.90181911,  0.98016918]), array([ 0.9862116 ,  1.37581682, -0.99191177,  0.21292891]), array([ 0.22174668,  1.49946952, -0.84184188,  0.29020765]), array([ 0.94606906,  1.72385406, -1.77260661,  0.10658114]), array([ 0.

In [9]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
pose

array([ 6.25234008e-01,  6.33557975e-01, -1.69977474e+00,  9.99314070e-01,
        6.61146104e-01,  5.39738536e-01, -1.62931979e+00,  9.98544157e-01,
        6.85576737e-01,  5.37809610e-01, -1.62955594e+00,  9.98731673e-01,
        7.05676675e-01,  5.36538124e-01, -1.62988889e+00,  9.97891426e-01,
        5.84320962e-01,  5.40723562e-01, -1.64942765e+00,  9.98680949e-01,
        5.57755291e-01,  5.41015685e-01, -1.64927161e+00,  9.98990119e-01,
        5.32287717e-01,  5.43705106e-01, -1.64963126e+00,  9.98697102e-01,
        7.25570381e-01,  5.55319190e-01, -1.16648042e+00,  9.98196065e-01,
        4.72773015e-01,  5.73468924e-01, -1.23379028e+00,  9.99262333e-01,
        6.60575986e-01,  7.23149598e-01, -1.51708651e+00,  9.99018431e-01,
        5.76737404e-01,  7.26078868e-01, -1.53939688e+00,  9.99383092e-01,
        8.63711357e-01,  9.71164286e-01, -8.71820092e-01,  9.74767327e-01,
        3.20785880e-01,  9.98131871e-01, -9.01819110e-01,  9.80169177e-01,
        9.86211598e-01,  

In [10]:
face = np.array([[res.x, res.y, res.z]
                for res in results.face_landmarks.landmark]).flatten()  if results.face_landmarks else np.zeros(468*3)

pose = np.array([[res.x, res.y, res.z, res.visibility]
                 for res in results.pose_landmarks.landmark]).flatten()  if results.pose_landmarks else np.zeros(33*4)

lh = np.array([[res.x, res.y, res.z]
               for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

rh = np.array([[res.x, res.y, res.z]
               for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [6]:
def extract_keypoints(results):
    face = np.array([[res.x, res.y, res.z]
                for res in results.face_landmarks.landmark]).flatten()  if results.face_landmarks else np.zeros(468*3)

    pose = np.array([[res.x, res.y, res.z, res.visibility]
                     for res in results.pose_landmarks.landmark]).flatten()  if results.pose_landmarks else np.zeros(33*4)
    
    lh = np.array([[res.x, res.y, res.z]
                   for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    rh = np.array([[res.x, res.y, res.z]
                   for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    return np.concatenate([pose,face,lh,rh])

In [7]:
np.shape(extract_keypoints(results))

NameError: name 'results' is not defined

## 4. Setup Folders for Collection

In [10]:
DATA_PATH = os.path.join('MP_DATA')                             # Path for exported data,  umpy array
actions = np.array(['hello', 'good', 'bad', 'beautiful'])       # Actions that we are truing to detect
no_sequences = 40                                               # Assume 40 videos
sequence_length = 40                                            # Assume 40 frame in a video

In [36]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## 5. Collect Keypoint Values for Training and Testing

In [37]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # loop over actions 
    for action in actions:
        # loop over videos
        for sequence in range(no_sequences):
            # loop over frames in a video
            for frame_num in range(sequence_length):
                
                ret, frame = cap.read()                                # read feed
                image, results = mediapipe_detection(frame, holistic)  # Make prediction
                draw_landmarks(image, results)                         # Draw landmarks

                # Apply wait and printing on screen logics  while video is on
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action,sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.imshow('Video to Detect', image)     # show to screen
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action,sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.imshow('Video to Detect', image)     # show to screen

                
                # Collecting keypoints or extract
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                if cv2.waitKey(10) & 0xFF==ord('q'):     # break gracefully
                    break
            
    cap.release()
    cv2.destroyAllWindows()

## 6. Preprocess Data and Create Labels and Features

In [16]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [17]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'hello': 0, 'good': 1, 'bad': 2, 'beautiful': 3}

In [20]:
sequences, labels = [], []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [21]:
np.array(sequences).shape

(160, 40, 1662)

In [22]:
np.array(labels).shape

(160,)

In [23]:
X = np.array(sequences)

In [24]:
y = to_categorical(labels).astype(int)

In [25]:
y

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1,

In [26]:
X_train, X_tes, y_train, y_test = train_test_split(X, y, test_size=0.1)

## 7. Build and Train LSTM Neural Network

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import TensorBoard

In [28]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [29]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(40,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

C:\Users\gdeva\.conda\envs\mediapipe_env\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [30]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 40, 64)              │         442,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 40, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4)                   │             132 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 596,708 (2.28 MB)

 Trainable params: 596,708 (2.28 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
a = [0.2,0.5,0.1, 0.2]
actions[np.argmax(a)]

'good'

In [32]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [33]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 178ms/step - categorical_accuracy: 0.2827 - loss: 44.0029
Epoch 2/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - categorical_accuracy: 0.2260 - loss: 105.5392
Epoch 3/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - categorical_accuracy: 0.2287 - loss: 125.6901
Epoch 4/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - categorical_accuracy: 0.2484 - loss: 297.7553
Epoch 5/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step - categorical_accuracy: 0.3058 - loss: 478.1408
Epoch 6/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 147ms/step - categorical_accuracy: 0.2345 - loss: 1630.4482
Epoch 7/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step - categorical_accuracy: 0.2444 - loss: 1918.3669
Epoch 8/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - categorical_accuracy: 0.2986 - loss: 976.5172
Epoch 9/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - categorical_accuracy: 0.2470 - loss: 583.0386
Epoch 10/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step - categorical_accuracy: 0.2643 - loss: 658.016

## 8. Make Predictions

In [38]:
res = model.predict(X_tes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 783ms/step


In [36]:
actions[np.argmax(res[4])]

'hello'

In [37]:
actions[np.argmax(y_test[4])]

'good'

## Saving the model

In [39]:
model.save('action.h5')

In [ ]:
del model
model.load_weights('action.h5')

## 10. Evaluation using Confusion Matrix and Accuracy

In [40]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [42]:
yhat = model.predict(X_tes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step


In [43]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [44]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[12,  0],
        [ 4,  0]],

       [[ 0, 11],
        [ 1,  4]],

       [[11,  1],
        [ 4,  0]],

       [[13,  0],
        [ 3,  0]]], dtype=int64)

In [45]:
accuracy_score(ytrue, yhat)

0.25

## Test 

In [46]:
colors = [(245,117,16), (117,245,16), (16,117,245), (19,250,44)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [48]:
# plt.figure(figsize=(18,18))
# plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'image' is not defined

<Figure size 1800x1800 with 0 Axes>

In [49]:
# sequence.reverse()

AttributeError: 'int' object has no attribute 'reverse'

In [ ]:
# len(sequence)

In [ ]:
# sequence.append('def')

In [ ]:
# sequence.reverse()

In [ ]:
# sequence[-30:]

In [50]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.6

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [51]:
cap.release()
cv2.destroyAllWindows()

In [52]:
res[np.argmax(res)] > threshold

True

In [53]:
(num_sequences,40,1662)

NameError: name 'num_sequences' is not defined

In [54]:
model.predict(np.expand_dims(X_tes[0], axis=0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step


array([[0., 0., 1., 0.]], dtype=float32)